<a href="https://colab.research.google.com/github/chiranthans23/hf-prompt-tutorial/blob/master/Push_to_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers evaluation datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for evaluation: filename=evaluation-0.0.2-py3-none-any.whl size=2479 sha256=68791a3b97be17cd0e381e714f6f014bd40a0e6475a2b6850898f7c6c742194b
  Stored in directo

In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

checkpoint = "bert-base-uncased"

model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [23]:
toks = tokenizer("I am not [MASK] as you think")
toks

{'input_ids': [101, 1045, 2572, 2025, 103, 2004, 2017, 2228, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [27]:
mask_token_index = torch.where(torch.tensor(toks["input_ids"]) == tokenizer.mask_token_id)
mask_token_index

(tensor([4]),)

In [28]:
import torch
logits = model(torch.tensor([toks['input_ids']]), attention_mask = torch.tensor([toks['attention_mask']])).logits
mask_token_logits = logits[0, mask_token_index, :]


In [29]:
top_3_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()

for token in top_3_tokens:
    print(tokenizer.decode([token]))

doing
quite
exactly


In [31]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [32]:
model.push_to_hub("my-test-mlm-model")
tokenizer.push_to_hub("my-test-mlm-tokenizer")

CommitInfo(commit_url='https://huggingface.co/chiranthans23/my-test-mlm-tokenizer/commit/bf0eac1ccecced121e8005fd26a65e99f4107200', commit_message='Upload tokenizer', commit_description='', oid='bf0eac1ccecced121e8005fd26a65e99f4107200', pr_url=None, pr_revision=None, pr_num=None)